In [37]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from string import punctuation
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import nltk
import pandas as pd
import seaborn as sns

### Define global constants
SEED = 42

### Getting data, modeling & separating into train/test data
df = pd.read_csv('data/imdb-reviews-pt-br.csv')

classification_column = df['sentiment'].replace(['neg', 'pos'], [0, 1])
df['classification'] = classification_column

### Define util global vars
all_words = ' '.join([text for text in df['text_pt']])
stopwords = nltk.corpus.stopwords.words('portuguese')
whitespace_tokenizer = nltk.tokenize.WhitespaceTokenizer()
punct_tokenizer = nltk.tokenize.WordPunctTokenizer() 

In [ ]:
def predict_text(text_df, x_column, y_column):
  vectorize = CountVectorizer(lowercase=False, max_features=50)
  bag_of_words = vectorize.fit_transform(text_df[x_column])

  x_train, x_test, y_train, y_test = train_test_split(
    bag_of_words,
    text_df[y_column],
    random_state=SEED
  )

  ### Predict with LogisticRegression
  logistic_regression_model = LogisticRegression()
  logistic_regression_model.fit(x_train, y_train)
  logistic_regression_accuracy = logistic_regression_model.score(x_test, y_test)

  print(logistic_regression_accuracy)

In [ ]:
predict_text(df, 'text_pt', 'classification')

In [ ]:
### Create a wordcloud
def get_words_wordcloud(text_df, x_column, y_column, y_value):
  all_words_based_on_y = text_df.query(f'{y_column} == "{y_value}"')
  all_words_based_on_y = ' '.join([text for text in all_words_based_on_y[x_column]])

  wordcloud = WordCloud(
    width=800, 
    height=500, 
    max_font_size=110,
    collocations=False
  ).generate(all_words_based_on_y)

  plt.figure(figsize=(10, 7))
  plt.imshow(wordcloud, interpolation='bilinear')
  plt.axis('off')
  plt.show()

In [ ]:
get_words_wordcloud(df, 'text_pt', 'sentiment', 'pos')
get_words_wordcloud(df, 'text_pt', 'sentiment', 'neg')

In [ ]:
### Tokenize white-spaces, getting words frequency & plot most frequent data
def tokenize_and_plot_most_frequent_data(text_df, x_column, quantity):
  all_words_from_x_column = ' '.join([text for text in text_df[x_column]])

  tokenized_phrases = whitespace_tokenizer.tokenize(all_words_from_x_column)
  words_frequency = nltk.FreqDist(tokenized_phrases)

  words_frequency_df = pd.DataFrame({ 
    'Word': list(words_frequency.keys()), 
    'Frequency': list(words_frequency.values()) 
  })

  n_most_frequent_words = words_frequency_df.nlargest(columns='Frequency', n=quantity)

  plt.figure(figsize=(12, 8))
  ax = sns.barplot(data=n_most_frequent_words, x='Word', y='Frequency', color='gray')
  ax.set(ylabel='Count')
  plt.show()

In [ ]:
tokenize_and_plot_most_frequent_data(df, 'text_pt', 10)

In [38]:
### Removes stopwords in all phrases from DataFrame
all_phrases_without_stopwords = []

for opinion in df['text_pt']:
  phrase_words = whitespace_tokenizer.tokenize(opinion)
  phrase_without_stopwords = []

  for word in phrase_words:
    if word not in stopwords:
      phrase_without_stopwords.append(word)

  all_phrases_without_stopwords.append(' '.join(phrase_without_stopwords))

df['filter_01'] = all_phrases_without_stopwords

In [ ]:
predict_text(df, 'filter_01', 'classification')

In [ ]:
tokenize_and_plot_most_frequent_data(df, 'filter_01', 10)

In [41]:
### Removes punctuation in all phrases from DataFrame
punctuation_list = [punct for punct in punctuation]
punctuation_and_stopwords = punctuation_list + stopwords

all_words_without_punctuation = []

for opinion in df['filter_01']:
  phrase_words = punct_tokenizer.tokenize(opinion)
  phrase_without_puncts = []

  for word in phrase_words:
    if word not in punctuation_and_stopwords:
      phrase_without_puncts.append(word)

  all_words_without_punctuation.append(' '.join(phrase_without_puncts))

df['filter_02'] = all_words_without_punctuation

,id,text_en,text_pt,sentiment,classification,filter_01,filter_02
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg,0,"Mais vez, Sr. Costner arrumou filme tempo nece...",Mais vez Sr Costner arrumou filme tempo necess...
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg,0,Este exemplo motivo maioria filmes ação mesmos...,Este exemplo motivo maioria filmes ação mesmos...
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg,0,"Primeiro tudo odeio raps imbecis, poderiam agi...",Primeiro tudo odeio raps imbecis poderiam agir...
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg,0,Nem Beatles puderam escrever músicas todos gos...,Nem Beatles puderam escrever músicas todos gos...
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg,0,"Filmes fotos latão palavra apropriada eles, ve...",Filmes fotos latão palavra apropriada verdade ...
